In [1]:
import json
from math import ceil
from pathlib import Path
from datasets import load_dataset
import pandas as pd

In [2]:
def dataframe_to_json(csv:str, save_path:str):
    df = pd.read_csv(csv)
    df = df[['prompt_kor', 'sql_context', 'sql']]
    
    with open(save_path, "w") as f:
        for row in df.iterrows():
            newitem = {
                "input": row[1]['prompt_kor'],
                "context": row[1]['sql_context'],
                "output": row[1]['sql'],
            }
            f.write(json.dumps(newitem) + "\n")

def get_train_val_splits(
    data_dir: str,
    val_ratio: float = 0,
    dpo_ratio: float = 0.2,
    seed: int = 42,
    shuffle: bool = True,
):
     
    def load_jsonl(data_dir):
        data_path = Path(data_dir).as_posix()
        data = load_dataset("json", data_files=data_path)
        return data

    data = load_jsonl(data_dir)
    num_samples = len(data["train"])
    val_set_size = ceil(val_ratio * num_samples)

    train_data_split = data["train"].train_test_split(
        # test_size=val_set_size,
        shuffle=shuffle, seed=seed
    )
    
    return train_data_split["train"].shuffle()
    # return train_data_split["train"].shuffle(), train_data_split["test"].shuffle()
    
    # dpo_set_size = ceil(dpo_ratio * len(train_data_split['test']))
    # dpo_data_split = train_data_split["test"].train_test_split(
    #     test_size=dpo_set_size,  shuffle=shuffle, seed=seed
    # )
    # return train_data_split["train"].shuffle(), dpo_data_split["train"].shuffle(), dpo_data_split["test"].shuffle()

def save_jsonl(data_dicts, out_path):
    with open(out_path, "w") as fp:
        for data_dict in data_dicts:
            fp.write(json.dumps(data_dict) + "\n")

In [3]:
data_csv = '../data/sql_context/sql-samples.csv'
json_name = '../data/sql_context/sql-samples.json'

In [4]:
dataframe_to_json(csv=data_csv, save_path=json_name)
sft_train_data = get_train_val_splits(data_dir=json_name) # 추후 excel 변환
save_jsonl(sft_train_data, "../data/sql_context/sft_train.jsonl")
# save_jsonl(dpo_train_data, "../data/sql_context/dpo_train.jsonl")
# save_jsonl(test_data, "../data/sql_context/test.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
# print(f"SFT Train: {len(sft_train_data)}\nDPO Train: {len(dpo_train_data)}\nTest: {len(test_data)}")
# print(f"SFT Train: {len(sft_train_data)}\nTest: {len(test_data)}")
len(sft_train_data)

10557

In [14]:
import pandas as pd
train_excel = pd.DataFrame(raw_train_data)
train_excel.to_excel('./data/health_train.xlsx')

val_excel = pd.DataFrame(raw_val_data)
val_excel.to_excel('./data/health_val.xlsx')